In [9]:
# ---
# jupyter:
#   jupytext:
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.11.5
#   kernelspec:
#     display_name: Python 3 (ipykernel)
#     language: python
#     name: python3
# ---

# +
import plotly.express as px
import plotly.graph_objects as go
import plotly as py
import pandas as pd
from plotly import tools
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash import Dash

###################################################################################################
app = Dash(__name__)
app.config['suppress_callback_exceptions'] = True

# Reading in data
df_plot3 = pd.read_csv("data/Superstore.csv")
# Removing outlier
df_plot3 = df_plot3.drop(df_plot3[df_plot3.Profit == -6599.9780].index)
# To sum profit and sales over all cities, for each state
df_plot3 = df_plot3.groupby(["State", "Category", "Sub-Category", "Ship Mode", "Segment"], as_index=False)[["Sales", "Profit","Discount"]].sum()
df_plot3 = pd.DataFrame(df_plot3)
# Adding column for profit margin
df_plot3['Profit Margin'] = df_plot3['Profit']/df_plot3['Sales']
                                      
def categorise(row):
    if row['Discount'] > 0:
        return 'Yes'
    return 'No'
df_plot3['Discount_Status'] = df_plot3.apply(lambda row: categorise(row), axis=1)
category_list = list(df_plot3["Category"].unique())
category_list.append("All Categories")
state_list = list(df_plot3["State"].unique())
state_list.append("All States") 
####################################################################################################
import dash_bootstrap_components as dbc
from dash import html
   
graph1 =  html.Div(
    [

    html.Br(),
    html.Br(),

    dcc.RadioItems(
        id='type-radio',
        options=[{'label': k, 'value': k} for k in ['Sales', 'Profit Margin']],
        value='Sales',
    ),
    html.Br(),
    html.Br(),
    
        
        
    html.Div(
    [html.Div(dcc.Dropdown(id='Category-dropdown',
                          options=[{'label': k, 'value': k} for k in category_list],
                        value='All Categories',
                            ), className='six columns',style={'width': '48%', 'display': 'inline-block'})  ,
     html.Div(dcc.Dropdown(
        id='state-dropdown',
      options=[{'label': k, 'value': k} for k in state_list],
        value='All States',
            ), 
              className='six columns',style={'width': '48%', 'display': 'inline-block'}) 
    
    ],
        
    style={'width': '48%', 'display': 'inline-block'},   
    className='container'
),
    
        
        
        
        
        

    html.Hr(),
    html.Br(),
    html.Br(),
    html.Div(id='display-selected-values'),
    dcc.Graph(id="mkt_graph"),
    ])




app.layout = graph1

# Make a figure based on the selections
@app.callback( # Columns 2m_temp_prod, or....
    Output('mkt_graph', 'figure'),
    [
     Input('Category-dropdown', 'value'),
    Input('state-dropdown', 'value'),
        
     Input('type-radio', 'value'),])
     
     
def make_graph(levels,states,type_graph):
    filter_list = ["Sub-Category","Category","Discount_Status"]
    if "All" not in states.split(" "):filter_list.append("State")
    if type_graph != "Profit Margin":
        group_by_profit = df_plot3.groupby(filter_list,as_index=False)[["Sales", "Profit Margin","Discount"]].sum()
    else :
        group_by_profit = df_plot3.groupby(filter_list,as_index=False)[["Sales","Profit Margin"]].mean()
        
        
        
        #group_by_profit["Profit Margin"] = group_by_profit["Profit Margin"]*100

    title_all =  "All Categories"
    state_all = "All states"
    
    if "All" not in levels.split(" "):
        title_all = levels
        group_by_profit = group_by_profit[group_by_profit["Category"]==levels]
    if "All" not in states.split(" "):
        state_all = states
        group_by_profit = group_by_profit[group_by_profit["State"]==states]
        
    if type_graph!="Profit Margin":
        fig = px.bar(group_by_profit, x="Sub-Category", y="Sales", color="Discount_Status", barmode='group', height=1000,color_discrete_sequence=["Red","midnightblue"])
        
        fig.update_layout(
            title={'text': "<b>Sales by "+str(title_all)+' in '+str(state_all)+"<b>",'y':1,'x':0.5,'xanchor': 'center','yanchor': 'top'},
            font_family="Calibri",
            font_size=20,
            font_color="black",
#            title_font_family="Times New Roman",
            title_font_color="black",
            legend_title_font_color="black",
            legend_title_font_size=20,
            legend_title = "Discount Applied?",
        )
        fig.update_xaxes()       
    else:
        
        
        fig = px.bar(group_by_profit, x="Sub-Category", y="Profit Margin", color="Discount_Status", barmode='group', height=1000 ,color_discrete_sequence=["Red","midnightblue"] )
        
        fig.update_layout(
            title={'text': "<b>Profit/Loss by "+str(title_all)+' in '+str(state_all)+' '+'<b>','y':1,'x':0.5,'xanchor': 'center','yanchor': 'top'},
            font_family="Calibri",
            font_size=20,
            font_color="black",
#            title_font_family="Times New Roman",
            title_font_color="black",
            legend_title_font_color="black",
            legend_title_font_size=20,
            legend_title = "Discount Applied?",
        
        )
        fig.update_xaxes(title={'text':str(levels)})
        
        fig.update_yaxes(tickformat=",.0%", title=None)
    fig.update_layout(
    legend=dict(
        x=1,
        y=1,
        traceorder="reversed",
#        title_font_family="Times New Roman",
        font=dict(
            family="Courier",
            size=12,
            color="black"
        ),
        bgcolor="lightgray",
        bordercolor="Black",
        borderwidth=2
    )
)
 
    
    return fig

# -

print("loaded all , now running the server")
#if __name__ == '__main__':
app.run_server(debug=False)



loaded all , now running the server
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Mar/2022 22:38:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2022 22:38:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2022 22:38:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2022 22:38:27] "GET /_favicon.ico?v=2.2.0 HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2022 22:38:27] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2022 22:38:27] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2022 22:38:27] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2022 22:38:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2022 22:38:27] "GET /_favicon.ico?v=2.2.0 HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2022 22:38:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2022 22:38:33] "POST /_dash-update-component HTTP/1.1" 200